In [1]:
# Initialize an additional dictionary to track last known mappings of Employee ID to Position ID
employee_position_map = {}

# Loop through each month's GHA data file
for month, gha_file in gha_files.items():
    # Load the GHA data for the current month
    gha_month_df = pd.read_csv(gha_file)
    gha_month_df = gha_month_df.rename(columns={'Position Number': 'Position ID', 'Employee ID': 'Employee ID_gha'})

    # Iterate over each row in the current month's GHA data
    for _, row in gha_month_df.iterrows():
        pos_id = row['Position ID']
        emp_id = row['Employee ID_gha']
        
        # Check for changes in Position ID -> Employee ID mapping
        if pos_id in position_employee_map:
            if position_employee_map[pos_id] != emp_id:
                change_record = master_df[master_df['Position ID'] == pos_id].copy()
                change_record['Change Month'] = month
                changes_df = pd.concat([changes_df, change_record], ignore_index=True)
                position_employee_map[pos_id] = emp_id
        else:
            position_employee_map[pos_id] = emp_id

        # Check for changes in Employee ID -> Position ID mapping
        if emp_id in employee_position_map:
            if employee_position_map[emp_id] != pos_id:
                change_record = master_df[master_df['Employee ID'] == emp_id].copy()
                change_record['Change Month'] = month
                changes_df = pd.concat([changes_df, change_record], ignore_index=True)
                employee_position_map[emp_id] = pos_id
        else:
            employee_position_map[emp_id] = pos_id

# Remove duplicate changes based on Position ID and Employee ID combinations
changes_df = changes_df.drop_duplicates(subset=['Position ID', 'Employee ID_gha'])

# View the final DataFrame with captured changes
print("Changes DataFrame:")
print(changes_df)


  Position ID Employee ID  Jan 24  Feb 24  Mar 24
0      000123        E001       1       1       0
1      000456        E002       1       0       0
2      000789        E003       0       0       1
